In [8]:
!pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 30.2 MB/s eta 0:00:0000:0100:01


In [2]:
from neo4j import GraphDatabase

uri = "neo4j+s://46a80122.databases.neo4j.io:7687"
user = "neo4j"
password = "abcde777"

driver = GraphDatabase.driver(uri, auth=(user, password))




In [3]:
def get_schema():
    with driver.session() as session:
        # Query node labels
        labels = session.run("CALL db.labels()").data()
        
        # Query relationship types
        relationships = session.run("CALL db.relationshipTypes()").data()
        
        # Query property keys
        properties = session.run("CALL db.propertyKeys()").data()
        
        # Get more detailed node information
        node_info = session.run("""
        MATCH (n)
        WITH DISTINCT labels(n) AS labels, keys(n) AS properties
        RETURN labels, properties
        """).data()
        
        # Get more detailed relationship information
        rel_info = session.run("""
        MATCH ()-[r]->()
        WITH DISTINCT type(r) AS type, keys(r) AS properties
        RETURN type, properties
        """).data()
        
        return {
            "labels": [label["label"] for label in labels],
            "relationships": [rel["relationshipType"] for rel in relationships],
            "properties": [prop["propertyKey"] for prop in properties],
            "node_details": node_info,
            "relationship_details": rel_info
        }




In [4]:
# Get the schema
schema = get_schema()
print(schema)



{'labels': ['Article', 'Code', 'Dataset', 'Software Application'], 'relationships': ['uses', 'provides', 'cites'], 'properties': ['id', 'data', 'name', 'nodes', 'relationships', 'style', 'visualisation', 'url', 'creativeWorkType', 'metadata_url'], 'node_details': [{'labels': ['Article'], 'properties': ['id', 'name', 'url', 'creativeWorkType', 'metadata_url']}, {'labels': ['Code'], 'properties': ['id', 'name', 'url', 'creativeWorkType', 'metadata_url']}, {'labels': ['Dataset'], 'properties': ['id', 'name', 'url', 'creativeWorkType', 'metadata_url']}, {'labels': ['Software Application'], 'properties': ['id', 'name', 'url', 'creativeWorkType', 'metadata_url']}], 'relationship_details': [{'type': 'uses', 'properties': []}, {'type': 'provides', 'properties': []}, {'type': 'cites', 'properties': []}]}


In [5]:
# Don't forget to close the driver when you're done
driver.close()

In [6]:
def schema_to_text(schema):
    text = "Database Schema:\n\n"
    text += "Node Labels: " + ", ".join(schema["labels"]) + "\n\n"
    text += "Relationship Types: " + ", ".join(schema["relationships"]) + "\n\n"
    text += "Properties: " + ", ".join(schema["properties"]) + "\n\n"
    
    text += "Node Details:\n"
    for node in schema["node_details"]:
        text += f"  Labels: {', '.join(node['labels'])}\n"
        text += f"  Properties: {', '.join(node['properties'])}\n\n"
    
    text += "Relationship Details:\n"
    for rel in schema["relationship_details"]:
        text += f"  Type: {rel['type']}\n"
        text += f"  Properties: {', '.join(rel['properties'])}\n\n"
    
    return text

schema_text = schema_to_text(schema)
print(schema_text)

Database Schema:

Node Labels: Article, Code, Dataset, Software Application

Relationship Types: uses, provides, cites

Properties: id, data, name, nodes, relationships, style, visualisation, url, creativeWorkType, metadata_url

Node Details:
  Labels: Article
  Properties: id, name, url, creativeWorkType, metadata_url

  Labels: Code
  Properties: id, name, url, creativeWorkType, metadata_url

  Labels: Dataset
  Properties: id, name, url, creativeWorkType, metadata_url

  Labels: Software Application
  Properties: id, name, url, creativeWorkType, metadata_url

Relationship Details:
  Type: uses
  Properties: 

  Type: provides
  Properties: 

  Type: cites
  Properties: 




In [9]:
from ctransformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "tomasonjo/text2cypher-demo-4bit-gguf",
    model_file="model.gguf",
    model_type="llama"
)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 0 files: 0it [00:00, ?it/s]

ValueError: Model file 'model.gguf' not found in '/Users/dhruvchandel/.cache/huggingface/hub/models--tomasonjo--text2cypher-demo-4bit-gguf/snapshots/456e5259ec3ef29946b0853ff76503bed8e5f331'